## MySQL

In [58]:
ucscDb <- dbConnect(MySQL(), user="genome", host="genome-mysql.cse.ucsc.edu")
result <- dbGetQuery(ucscDb, "show databases;");
#dbDisconnect(ucscDb);
nrow(result)

[1] 205

In [59]:
hg19 <- dbConnect(MySQL(), user="genome", db="hg19", host="genome-mysql.cse.ucsc.edu")
class(hg19)
allTables <- dbListTables(hg19)
length(allTables)
allTables[1:5]
dbDisconnect(hg19)

[1] "MySQLConnection"
attr(,"package")
[1] "RMySQL"

[1] 11025

[1] "HInv"         "HInvGeneMrna" "acembly"      "acemblyClass" "acemblyPep"

[1] TRUE

In [60]:
dbListFields(hg19, "affyU133Plus2")
dbGetQuery(hg19, "select count(*) from affyU133Plus2")

# not working dont know why
affyData <- dbReadTable(hg19, "affyU133Plus2")
head(affyData)
query <- dbSendQuery(hg19, "select * from affyU133Plus2 where misMatches between 1 and 3")
affyMiss <- fetch(query); quantile(affyMiss$misMatches)

# selecting top 10 rows
affyMisSmall <- fetch(query, n=10); dbClearResult(query); # clearing query 
dbDisconnect(hg19)

ERROR: Error in .local(dbObj, ...): internal error in RS_DBI_getConnection: corrupt connection handle


ERROR: Error in .local(dbObj, ...): internal error in RS_DBI_getConnection: corrupt connection handle


ERROR: Error in .local(dbObj, ...): internal error in RS_DBI_getConnection: corrupt connection handle


ERROR: Error in head(affyData): object 'affyData' not found


ERROR: Error in .local(dbObj, ...): internal error in RS_DBI_getConnection: corrupt connection handle


ERROR: Error in mysqlFetch(res, n = 0, ...): INTEGER() can only be applied to a 'integer', not a 'character'


ERROR: Error in quantile(affyMiss$misMatches): object 'affyMiss' not found


ERROR: Error in is(object, Cl): INTEGER() can only be applied to a 'integer', not a 'character'


ERROR: Error in .local(dbObj, ...): INTEGER() can only be applied to a 'integer', not a 'character'


ERROR: Error in .local(dbObj, ...): internal error in RS_DBI_getConnection: corrupt connection handle


In [61]:
dbDisconnect(hg19)

ERROR: Error in .local(dbObj, ...): internal error in RS_DBI_getConnection: corrupt connection handle


## HDF5

In [62]:
#source("http://bioconductor.org/biocLite.R")
#biocLite("rhdf5")


The downloaded binary packages are in
	C:\Users\Sid\AppData\Local\Temp\RtmpCi2VK7\downloaded_packages


Bioconductor version 3.0 (BiocInstaller 1.16.5), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: http://bioconductor.org
Using Bioconductor version 3.0 (BiocInstaller 1.16.5), R version 3.1.3.
Installing package(s) 'rhdf5'
also installing the dependency 'zlibbioc'



package 'zlibbioc' successfully unpacked and MD5 sums checked
package 'rhdf5' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Sid\AppData\Local\Temp\RtmpCi2VK7\downloaded_packages


Old packages: 'base64enc', 'BH', 'boot', 'car', 'caret', 'chron', 'class',
  'cluster', 'codetools', 'colorspace', 'data.table', 'digest', 'dplyr',
  'evaluate', 'foreach', 'foreign', 'formatR', 'ggplot2', 'gtools', 'hexbin',
  'highr', 'htmltools', 'httpuv', 'iterators', 'jsonlite', 'KernSmooth',
  'knitr', 'lattice', 'lme4', 'maps', 'MASS', 'Matrix', 'mgcv', 'mime', 'nlme',
  'nnet', 'pbkrtest', 'plyr', 'quantreg', 'R6', 'randomForest', 'Rcpp',
  'RcppEigen', 'rmarkdown', 'rpart', 'scales', 'shiny', 'SparseM', 'spatial',
  'stringi', 'stringr', 'survival', 'tidyr', 'uuid', 'xtable', 'zoo'


In [63]:
library(rhdf5)

In [64]:
#created = h5createFile("example.h5")
#created

[1] TRUE

In [73]:
created = h5createGroup("example.h5", "foo")
created = h5createGroup("example.h5", "baa")
created = h5createGroup("example.h5", "foo/foobaa")
h5ls("example.h5")

,group,name,otype,dclass,dim
0,/,baa,H5I_GROUP,,
1,/,foo,H5I_GROUP,,
2,/foo,foobaa,H5I_GROUP,,


In [74]:
A <- matrix(1:10, nr = 5, nc = 2)
h5write(A, "example.h5", "foo/A")
B = array(seq(0.1, 2, by = 0.1), dim = c(5, 2, 2))
attr(B, "scale") <- "liter"
h5write(B, "example.h5", "foo/foobaa/B")
h5ls("example.h5")

,group,name,otype,dclass,dim
0,/,baa,H5I_GROUP,,
1,/,foo,H5I_GROUP,,
2,/foo,A,H5I_DATASET,INTEGER,5 x 2
3,/foo,foobaa,H5I_GROUP,,
4,/foo/foobaa,B,H5I_DATASET,FLOAT,5 x 2 x 2


In [75]:
# The following example creates a data frame and writes it to the top of the HDF5 file's hierarchy, using the name “df”.

df = data.frame(1L:5L, seq(0, 1, length.out = 5), c("ab", "cde", "fghi", "a", 
    "s"), stringsAsFactors = F)
h5write(df, "example.h5", "df")
h5ls("example.h5")


,group,name,otype,dclass,dim
0,/,baa,H5I_GROUP,,
1,/,df,H5I_DATASET,COMPOUND,5
2,/,foo,H5I_GROUP,,
3,/foo,A,H5I_DATASET,INTEGER,5 x 2
4,/foo,foobaa,H5I_GROUP,,
5,/foo/foobaa,B,H5I_DATASET,FLOAT,5 x 2 x 2


In [76]:
readA = h5read("example.h5", "foo/A")
readB = h5read("example.h5", "foo/foobaa/B")
readdf = h5read("example.h5", "df")
readA

1,6
2,7
3,8
4,9
5,10


In [77]:
foo = h5read("example.h5", "foo")
class(foo)
names(foo)
class(foo$foobaa)

[1] "list"

[1] "A"      "foobaa"

[1] "list"

In [78]:
h5write(c(12, 13, 14), "example.h5", "foo/A", index = list(1:3, 1))
h5read("example.h5", "foo/A")

12,6
13,7
14,8
4,9
5,10


## Web

In [85]:
con <- url("http://scholar.google.com/citations?user=HI-I6C0AAAAJ&l=en")
htmlCode <- readLines(con)
close(con)
#substr(htmlCode, start = 1, stop = 1000)
#htmlCode
class(htmlCode)
length(htmlCode)
nchar(htmlCode)

Warning message:
In readLines(con): incomplete final line found on 'http://scholar.google.com/citations?user=HI-I6C0AAAAJ&l=en'

[1] "character"

[1] 1

[1] 97526

In [96]:
#install.packages("XML", repos = "http://cran.us.r-project.org")
library(XML)
url <- "http://scholar.google.com/citations?user=HI-I6C0AAAAJ&hl=en"
html <- htmlTreeParse(url, useInternalNodes = T)
xpathSApply(html, "//title", xmlValue)
xpathSApply(html, "//td[@id='col-citedby']", xmlValue)

ERROR: Error: failed to load external entity "http://scholar.google.com/citations?user=HI-I6C0AAAAJ&hl=en"



[1] "Jeff Leek - Google Scholar Citations"

list()

In [97]:
#install.packages("httr", repos = "http://cran.us.r-project.org")
#library(httr)
html2 = GET(url)
class(html2)
content2 = content(html2, as = "text")
class(content2)
parsedHtml = htmlParse(content2, asText = TRUE)
xpathSApply(parsedHtml, "//title", xmlValue)

[1] "response"

[1] "character"

[1] "Jeff Leek - Google Scholar Citations"